In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column
from tqdm import tqdm

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_offset = get_files('table_fits_bgcorr.fits', root_dir)
files_slope = get_files('table_fits_NII.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/table_fits_bgcorr.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/dat

In [5]:
gals  = []
output_stats = []

for file_offset, file_slope in tqdm(zip(files_offset, files_slope)): 

    # Loading files...
    table_offset = QTable.read(file_offset)
    table_slope = QTable.read(file_slope)

    gal1 = file_offset.split('/')[-3]
    gal2 = file_offset.split('/')[-3]

    if gal1 != gal2: 
        print('ERROR - galaxies do not match!')
        break

    gals += [gal1]

    intercepts = np.float32(table_offset['intercept_lowres'].value)
    intercept1 = intercepts[np.where(table_offset['filter'] == 'broad1')[0][0]]
    intercept2 = intercepts[np.where(table_offset['filter'] == 'narrow')[0][0]]
    intercept3 = intercepts[np.where(table_offset['filter'] == 'broad2')[0][0]]

    slope = np.float32(table_slope['slope_bins'][0]) # for anchored image only

    stats = np.array([intercept1, intercept2, intercept3, slope])
    output_stats += [stats]

gals = np.array(gals, dtype='<U8')
output_stats = np.array(output_stats)

20it [00:00, 123.70it/s]


In [6]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    # if '628C' in gal:
    #     gal_split = gal.split('628C')
    #     gal = gal_split[0] + '0628'
    gal_names[i] = gal

names=('F555W', 'F65XN', 'F814W', 'N[II] corr')

table1 = Column(gal_names, name='Galaxy')
table2 = QTable(output_stats, names=names)

table = hstack([table1, table2])
table.sort('Galaxy')

# Convert from per pixel to per arcsec^2 
# Currently the pixel size is 0.2 arcsec - MUSE pixel size

pix_size = 0.2 *u.arcsec
pix_size = pix_size.to('arcsec')
pix_area = pix_size**2
unit = u.erg/u.s/u.cm**2/u.A

table['F555W'] = table['F555W']/pix_area*unit
table['F65XN'] = table['F65XN']/pix_area*unit
table['F814W'] = table['F814W']/pix_area*unit

table

Galaxy,F555W,F65XN,F814W,N[II] corr
,erg / (A s arcsec2 cm2),erg / (A s arcsec2 cm2),erg / (A s arcsec2 cm2),
str8,float32,float32,float32,float32
IC~5332,-290.7807312011719,1801.1290283203125,-190.28854370117188,1.1107948
NGC~1087,-217.994140625,886.8765869140625,-147.06248474121094,1.1829741
NGC~1300,-84.712158203125,-92.36585998535156,-51.87118911743164,1.2616084
NGC~1365,-777.3173217773438,520.9389038085938,-687.0180053710938,1.2520517
NGC~1385,-242.33731079101562,890.5015258789062,-164.82569885253906,1.213685
NGC~1433,-617.6177368164062,-620.5584106445312,-534.8125610351562,1.3250552
NGC~1512,-161.9374237060547,1414.8905029296875,-140.04957580566406,1.0318233
NGC~1566,-1030.7138671875,1660.5478515625,-575.8243408203125,1.0282753


In [7]:
formats={
        'Galaxy': '%s',
        'F555W': '%.2f',
        'F65XN': '%.2f',
        'F814W': '%.2f',
        'N[II] corr': '%.2f'
        }

table.write('./tabs/galfit_props.tex', formats=formats, overwrite=True)

In [8]:
np.nanmean(table['N[II] corr'])

1.1580018

In [9]:
gals  = []
output_stats = []

for file_offset, file_slope in tqdm(zip(files_offset, files_slope)): 

    # Loading files...
    table_offset = QTable.read(file_offset)
    table_slope = QTable.read(file_slope)

    gal1 = file_offset.split('/')[-3]
    gal2 = file_offset.split('/')[-3]

    if gal1 != gal2: 
        print('ERROR - galaxies do not match!')
        break

    gals += [gal1]

    intercepts = np.float32(table_offset['interceptfixed_bins'].value)
    intercept1 = intercepts[np.where(table_offset['filter'] == 'broad1')[0][0]]
    intercept2 = intercepts[np.where(table_offset['filter'] == 'narrow')[0][0]]
    intercept3 = intercepts[np.where(table_offset['filter'] == 'broad2')[0][0]]

    slope = np.float32(table_slope['slopefixed_bins'][0]) # for anchored image only

    stats = np.array([intercept1, intercept2, intercept3, slope])
    output_stats += [stats]

gals = np.array(gals, dtype='<U8')
output_stats = np.array(output_stats)

20it [00:00, 116.21it/s]


In [10]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    # if '628C' in gal:
    #     gal_split = gal.split('628C')
    #     gal = gal_split[0] + '0628'
    gal_names[i] = gal

names=('F555W fix', 'F65XN fix', 'F814W fix', 'N[II] corr fix')

table1 = Column(gal_names, name='Galaxy')
table2 = QTable(output_stats, names=names)

table = hstack([table1, table2])
table.sort('Galaxy')

# Convert from per pixel to per arcsec^2 
# Currently the pixel size is 0.2 arcsec - MUSE pixel size

pix_size = 0.2 *u.arcsec
pix_size = pix_size.to('arcsec')
pix_area = pix_size**2
unit = u.erg/u.s/u.cm**2/u.A


table['F555W fix'] = table['F555W fix']/pix_area*unit
table['F65XN fix'] = table['F65XN fix']/pix_area*unit
table['F814W fix'] = table['F814W fix']/pix_area*unit

table

Galaxy,F555W fix,F65XN fix,F814W fix,N[II] corr fix
,erg / (A s arcsec2 cm2),erg / (A s arcsec2 cm2),erg / (A s arcsec2 cm2),
str8,float32,float32,float32,float32
IC~5332,-337.9557189941406,1838.069580078125,-227.2178955078125,1.0967219
NGC~1087,-272.1636047363281,828.0471801757812,-181.56982421875,1.1880786
NGC~1300,-51.867958068847656,-94.1435775756836,-60.14449691772461,1.3258827
NGC~1365,-882.4429931640625,438.5127258300781,-723.0433959960938,1.2690622
NGC~1385,-249.11956787109375,832.7805786132812,-190.40879821777344,1.2314469
NGC~1433,-620.6744995117188,-663.5328979492188,-544.6904296875,1.4316695
NGC~1512,-198.7041778564453,1435.6768798828125,-128.09605407714844,1.0282103
NGC~1566,-1022.7232055664062,1544.006591796875,-577.02587890625,1.0300295


In [11]:
formats={
        'Galaxy': '%s',
        'F555W fix': '%.2f',
        'F65XN fix': '%.2f',
        'F814W fix': '%.2f',
        'N[II] corr fix': '%.2f'
        }

table.write('./tabs/galfitfixed_props.tex', formats=formats, overwrite=True)